## Setup and Imports

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from pathlib import Path
import time
import matplotlib.pyplot as plt
from ogb.nodeproppred import NodePropPredDataset

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

Using device: cuda
GPU: NVIDIA GeForce RTX 3060


## Load Dataset

Loading the ogbn-products dataset from cache (already downloaded in first iteration).

In [9]:
print("Loading ogbn-products dataset...")

import os
import shutil
from ogb.nodeproppred import NodePropPredDataset

dataset_root = '../dataset'
processed_file = os.path.join(dataset_root, 'ogbn_products', 'processed', 'data_processed')

# Delete the locked processed file to force fresh processing
if os.path.exists(processed_file):
    try:
        print(f"Removing locked processed file: {processed_file}")
        os.remove(processed_file)
        print("  File removed successfully")
    except PermissionError:
        print("  File is locked - trying to work around it...")
        # Rename processed folder to bypass the lock
        processed_dir = os.path.join(dataset_root, 'ogbn_products', 'processed')
        backup_dir = os.path.join(dataset_root, 'ogbn_products', 'processed_backup')
        
        try:
            if os.path.exists(backup_dir):
                shutil.rmtree(backup_dir)
            os.rename(processed_dir, backup_dir)
            print(f"  Moved processed folder to backup")
        except Exception as e2:
            print(f"  Could not rename: {e2}")
            print("  Trying alternative: loading with temporary directory...")
            
            # Last resort: use completely separate temp directory
            import tempfile
            temp_root = tempfile.mkdtemp()
            print(f"  Using temporary directory: {temp_root}")
            dataset = NodePropPredDataset(name='ogbn-products', root=temp_root)
            graph, labels_raw = dataset[0]
            node_features = torch.tensor(graph['node_feat'], dtype=torch.float)
            edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)
            labels = torch.tensor(labels_raw, dtype=torch.long).squeeze()
            split_idx = dataset.get_idx_split()
            
            # Skip to the end
            train_idx = torch.tensor(split_idx['train'], dtype=torch.long)
            val_idx = torch.tensor(split_idx['valid'], dtype=torch.long)
            test_idx = torch.tensor(split_idx['test'], dtype=torch.long)
            
            num_nodes = node_features.shape[0]
            num_features = node_features.shape[1]
            num_classes = len(torch.unique(labels))
            
            print(f"\nDataset Statistics:")
            print(f"  Nodes: {num_nodes:,}")
            print(f"  Edges: {edge_index.shape[1]:,}")
            print(f"  Features: {num_features}")
            print(f"  Classes: {num_classes}")
            print(f"  Train nodes: {train_idx.shape[0]:,}")
            print(f"  Val nodes: {val_idx.shape[0]:,}")
            print(f"  Test nodes: {test_idx.shape[0]:,}")
            
            degree = torch.bincount(edge_index[0])
            cold_start_nodes = (degree <= 2).nonzero(as_tuple=True)[0]
            print(f"\nCold-start nodes (degree ≤ 2): {len(cold_start_nodes):,}")
            
            # Exit the cell here - we're done
            raise SystemExit(0)

# If we get here, the file was removed or renamed successfully
print("Loading dataset with OGB...")
dataset = NodePropPredDataset(name='ogbn-products', root=dataset_root)
graph, labels_raw = dataset[0]

node_features = torch.tensor(graph['node_feat'], dtype=torch.float)
edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)
labels = torch.tensor(labels_raw, dtype=torch.long).squeeze()
split_idx = dataset.get_idx_split()

# Get splits
train_idx = torch.tensor(split_idx['train'], dtype=torch.long)
val_idx = torch.tensor(split_idx['valid'], dtype=torch.long)
test_idx = torch.tensor(split_idx['test'], dtype=torch.long)

# Dataset statistics
num_nodes = node_features.shape[0]
num_features = node_features.shape[1]
num_classes = len(torch.unique(labels))

print(f"\nDataset Statistics:")
print(f"  Nodes: {num_nodes:,}")
print(f"  Edges: {edge_index.shape[1]:,}")
print(f"  Features: {num_features}")
print(f"  Classes: {num_classes}")
print(f"  Train nodes: {train_idx.shape[0]:,}")
print(f"  Val nodes: {val_idx.shape[0]:,}")
print(f"  Test nodes: {test_idx.shape[0]:,}")

# Compute degree for cold-start analysis
degree = torch.bincount(edge_index[0])
cold_start_nodes = (degree <= 2).nonzero(as_tuple=True)[0]
print(f"\nCold-start nodes (degree ≤ 2): {len(cold_start_nodes):,}")

Loading ogbn-products dataset...
Removing locked processed file: ../dataset\ogbn_products\processed\data_processed
  File is locked - trying to work around it...
  Moved processed folder to backup
Loading dataset with OGB...


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [02:39<00:00,  8.87it/s]



Extracting ../dataset\products.zip
Loading necessary files...
This might take a while.
Loading necessary files...
This might take a while.
Processing graphs...
Processing graphs...


100%|██████████| 1/1 [00:01<00:00,  1.14s/it]



Saving...

Dataset Statistics:
  Nodes: 2,449,029
  Edges: 123,718,280
  Features: 100
  Classes: 47
  Train nodes: 196,615
  Val nodes: 39,323
  Test nodes: 2,213,091

Dataset Statistics:
  Nodes: 2,449,029
  Edges: 123,718,280
  Features: 100
  Classes: 47
  Train nodes: 196,615
  Val nodes: 39,323
  Test nodes: 2,213,091

Cold-start nodes (degree ≤ 2): 236,775

Cold-start nodes (degree ≤ 2): 236,775


## Build Adjacency List for Neighbor Sampling

In [10]:
print("Building adjacency list for neighbor sampling...")

adj_list = {i: [] for i in range(num_nodes)}
for src, dst in edge_index.t().numpy():
    adj_list[src].append(dst)
    adj_list[dst].append(src)  # undirected graph

print("Adjacency list created")

Building adjacency list for neighbor sampling...
Adjacency list created
Adjacency list created


## Helper Functions for Neighbor Sampling

In [11]:
def sample_neighbors(nodes, adj_list, num_samples=[10, 5]):
    """Sample neighbors for multiple hops"""
    all_nodes = set(nodes)
    current_layer = set(nodes)
    edges = []
    
    for k in num_samples:
        next_layer = set()
        for node in current_layer:
            neighbors = adj_list.get(node, [])
            if len(neighbors) > 0:
                sampled = np.random.choice(
                    neighbors, 
                    size=min(k, len(neighbors)), 
                    replace=False
                )
                for neighbor in sampled:
                    edges.append((node, neighbor))
                    next_layer.add(neighbor)
                    all_nodes.add(neighbor)
        current_layer = next_layer
    
    return list(all_nodes), edges


def create_batches(node_indices, batch_size, adj_list, num_samples=[10, 5]):
    """Generate mini-batches with sampled neighborhoods"""
    indices = node_indices.cpu().numpy()
    np.random.shuffle(indices)
    
    batches = []
    for i in range(0, len(indices), batch_size):
        batch_nodes = indices[i:i+batch_size]
        sampled_nodes, sampled_edges = sample_neighbors(batch_nodes.tolist(), adj_list, num_samples)
        batches.append((batch_nodes, sampled_nodes, sampled_edges))
    
    return batches

print("Neighbor sampling functions defined")

Neighbor sampling functions defined


## Model Architectures

### 1. MLP Baseline (No Graph Structure)

In [12]:
class MLP(nn.Module):
    """Multi-Layer Perceptron baseline - ignores graph structure"""
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=3, dropout=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.layers.append(nn.Linear(hidden_channels, hidden_channels))
        self.layers.append(nn.Linear(hidden_channels, out_channels))
    
    def forward(self, x, edge_index=None):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

print("MLP model defined")

MLP model defined


### 2. GraphSAGE with Mean Aggregation

In [13]:
class SAGEConvLayer(nn.Module):
    """GraphSAGE layer with mean aggregation"""
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(2 * in_features, out_features)
    
    def forward(self, x, edge_index):
        row, col = edge_index
        
        # Mean aggregation
        agg = torch.zeros(x.size(0), x.size(1), device=x.device)
        deg = torch.zeros(x.size(0), device=x.device)
        
        agg.index_add_(0, row, x[col])
        deg.index_add_(0, row, torch.ones(row.size(0), device=x.device))
        
        deg = deg.clamp(min=1).unsqueeze(1)
        agg = agg / deg
        
        # Concatenate and transform
        out = torch.cat([x, agg], dim=1)
        out = self.linear(out)
        return out


class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        
        self.convs.append(SAGEConvLayer(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConvLayer(hidden_channels, hidden_channels))
        self.convs.append(SAGEConvLayer(hidden_channels, out_channels))
    
    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

print("GraphSAGE model defined")

GraphSAGE model defined


### 3. GraphSAGE with Max Pooling

In [23]:
class SAGEMaxPoolLayer(nn.Module):
    """GraphSAGE layer with max pooling aggregation"""
    def __init__(self, in_features, out_features):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU()
        )
        self.linear = nn.Linear(2 * in_features, out_features)
    
    def forward(self, x, edge_index):
        row, col = edge_index
        
        # Transform neighbor features
        neighbor_feats = self.mlp(x)
        
        # Max pooling aggregation - avoid in-place operations
        agg = torch.full((x.size(0), x.size(1)), float('-inf'), device=x.device)
        
        # Use scatter_reduce for max pooling (no in-place modification)
        if edge_index.size(1) > 0:
            agg = torch.scatter_reduce(
                agg, 0, row.unsqueeze(1).expand(-1, x.size(1)), 
                neighbor_feats[col], reduce='amax', include_self=False
            )
        
        # Replace -inf with 0 for nodes without neighbors
        agg = torch.where(torch.isinf(agg), torch.zeros_like(agg), agg)
        
        # Concatenate and transform
        out = torch.cat([x, agg], dim=1)
        out = self.linear(out)
        return out


class GraphSAGEMaxPool(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        
        self.convs.append(SAGEMaxPoolLayer(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEMaxPoolLayer(hidden_channels, hidden_channels))
        self.convs.append(SAGEMaxPoolLayer(hidden_channels, out_channels))
    
    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

print("GraphSAGE MaxPool model defined")

GraphSAGE MaxPool model defined


### 4. Graph Convolutional Network (GCN)

In [24]:
class GCNLayer(nn.Module):
    """Graph Convolutional Network layer"""
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, x, edge_index):
        row, col = edge_index
        
        # Compute degree normalization
        deg = torch.zeros(x.size(0), device=x.device)
        deg.index_add_(0, row, torch.ones(row.size(0), device=x.device))
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        
        # Normalize features
        x = self.linear(x)
        
        # Aggregate neighbors with normalization
        out = torch.zeros_like(x)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        for i in range(edge_index.size(1)):
            out[row[i]] += norm[i] * x[col[i]]
        
        return out


class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        
        self.convs.append(GCNLayer(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GCNLayer(hidden_channels, hidden_channels))
        self.convs.append(GCNLayer(hidden_channels, out_channels))
    
    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

print("GCN model defined")

GCN model defined


### 5. Graph Attention Network (GAT)

In [25]:
class GATLayer(nn.Module):
    """Graph Attention Network layer"""
    def __init__(self, in_features, out_features, dropout=0.5):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        
        self.W = nn.Linear(in_features, out_features, bias=False)
        self.a = nn.Parameter(torch.zeros(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
    
    def forward(self, x, edge_index):
        row, col = edge_index
        
        # Linear transformation
        h = self.W(x)
        
        # Attention mechanism
        a_input = torch.cat([h[row], h[col]], dim=1)
        e = F.leaky_relu(torch.matmul(a_input, self.a).squeeze(), negative_slope=0.2)
        
        # Compute attention coefficients
        attention = torch.zeros(x.size(0), device=x.device)
        attention.index_add_(0, row, torch.exp(e))
        
        alpha = torch.exp(e) / (attention[row] + 1e-16)
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        # Aggregate with attention
        out = torch.zeros(x.size(0), self.out_features, device=x.device)
        for i in range(edge_index.size(1)):
            out[row[i]] += alpha[i] * h[col[i]]
        
        return out


class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        
        self.convs.append(GATLayer(in_channels, hidden_channels, dropout))
        for _ in range(num_layers - 2):
            self.convs.append(GATLayer(hidden_channels, hidden_channels, dropout))
        self.convs.append(GATLayer(hidden_channels, out_channels, dropout))
    
    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < self.num_layers - 1:
                x = F.elu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

print("GAT model defined")

GAT model defined


## Training and Evaluation Functions

In [26]:
def train_epoch_mlp(model, train_idx, batch_size, node_feats, node_labels, optimizer, loss_fn, device):
    """Training function for MLP (no neighbor sampling needed)"""
    model.train()
    total_loss = 0
    total_examples = 0
    
    indices = train_idx.cpu().numpy()
    np.random.shuffle(indices)
    
    for i in range(0, len(indices), batch_size):
        batch_idx = indices[i:i+batch_size]
        optimizer.zero_grad()
        
        batch_x = node_feats[batch_idx].to(device)
        batch_y = node_labels[batch_idx].to(device)
        
        out = model(batch_x)
        loss = loss_fn(out, batch_y)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(batch_idx)
        total_examples += len(batch_idx)
    
    return total_loss / max(total_examples, 1)


def train_epoch_gnn(model, train_idx, batch_size, adj_list, node_feats, node_labels, 
                    optimizer, loss_fn, device, num_samples=[10, 5]):
    """Training function for GNN models with neighbor sampling"""
    model.train()
    total_loss = 0
    total_examples = 0
    
    batches = create_batches(train_idx, batch_size, adj_list, num_samples)
    
    for batch_target_nodes, sampled_nodes, sampled_edges in batches:
        optimizer.zero_grad()
        
        node_map = {n: i for i, n in enumerate(sampled_nodes)}
        batch_x = node_feats[sampled_nodes].to(device)
        batch_y = node_labels[batch_target_nodes].to(device)
        
        # Create subgraph
        subgraph_edges = []
        for src, dst in sampled_edges:
            if src in node_map and dst in node_map:
                subgraph_edges.append([node_map[src], node_map[dst]])
        
        if len(subgraph_edges) > 0:
            batch_edge_index = torch.tensor(subgraph_edges, dtype=torch.long).t().to(device)
        else:
            batch_edge_index = torch.empty((2, 0), dtype=torch.long, device=device)
        
        out = model(batch_x, batch_edge_index)
        target_indices = [node_map[n] for n in batch_target_nodes if n in node_map]
        loss = loss_fn(out[target_indices], batch_y[:len(target_indices)])
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(target_indices)
        total_examples += len(target_indices)
    
    return total_loss / max(total_examples, 1)


@torch.no_grad()
def evaluate_mlp(model, node_feats, node_labels, mask, device, batch_size=2048):
    """Evaluation for MLP"""
    model.eval()
    eval_indices = mask.nonzero(as_tuple=True)[0].numpy()
    
    all_preds = []
    all_labels = []
    
    for i in range(0, len(eval_indices), batch_size):
        batch_idx = eval_indices[i:i+batch_size]
        batch_x = node_feats[batch_idx].to(device)
        out = model(batch_x)
        pred = out.argmax(dim=1).cpu()
        all_preds.append(pred)
        all_labels.append(node_labels[batch_idx])
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    acc = (all_preds == all_labels).sum().item() / len(all_labels)
    return acc


@torch.no_grad()
def evaluate_gnn(model, node_feats, node_labels, mask, device, adj_list, 
                 batch_size=2048, num_samples=[10, 5]):
    """Evaluation for GNN models"""
    model.eval()
    eval_indices = mask.nonzero(as_tuple=True)[0].numpy()
    
    all_preds = []
    all_labels = []
    
    for i in range(0, len(eval_indices), batch_size):
        batch_target_nodes = eval_indices[i:i+batch_size]
        sampled_nodes, sampled_edges = sample_neighbors(batch_target_nodes.tolist(), adj_list, num_samples)
        node_map = {n: idx for idx, n in enumerate(sampled_nodes)}
        
        batch_x = node_feats[sampled_nodes].to(device)
        
        subgraph_edges = []
        for src, dst in sampled_edges:
            if src in node_map and dst in node_map:
                subgraph_edges.append([node_map[src], node_map[dst]])
        
        if len(subgraph_edges) > 0:
            batch_edge_index = torch.tensor(subgraph_edges, dtype=torch.long).t().to(device)
        else:
            batch_edge_index = torch.empty((2, 0), dtype=torch.long, device=device)
        
        out = model(batch_x, batch_edge_index)
        target_indices = [node_map[n] for n in batch_target_nodes if n in node_map]
        pred = out[target_indices].argmax(dim=1).cpu()
        all_preds.append(pred)
        all_labels.append(node_labels[batch_target_nodes[:len(target_indices)]])
        
        del out, batch_x, batch_edge_index
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    acc = (all_preds == all_labels).sum().item() / len(all_labels)
    return acc

print("Training and evaluation functions defined")

Training and evaluation functions defined


## Model Training Wrapper

In [27]:
def train_model(model_name, model, is_gnn=True, epochs=10, batch_size=1024, lr=0.001):
    """Train a model and track performance"""
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    
    # Create masks
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[train_idx.cpu()] = True
    val_mask[val_idx.cpu()] = True
    test_mask[test_idx.cpu()] = True
    
    history = {
        'train_loss': [],
        'val_acc': [],
        'test_acc': [],
        'epoch_time': []
    }
    
    best_val_acc = 0
    best_test_acc = 0
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Train
        if is_gnn:
            loss = train_epoch_gnn(model, train_idx, batch_size, adj_list, 
                                   node_features.cpu(), labels.cpu(), 
                                   optimizer, loss_fn, device)
        else:
            loss = train_epoch_mlp(model, train_idx, batch_size, 
                                   node_features.cpu(), labels.cpu(), 
                                   optimizer, loss_fn, device)
        
        # Evaluate
        if is_gnn:
            val_acc = evaluate_gnn(model, node_features.cpu(), labels.cpu(), 
                                   val_mask, device, adj_list)
            test_acc = evaluate_gnn(model, node_features.cpu(), labels.cpu(), 
                                    test_mask, device, adj_list)
        else:
            val_acc = evaluate_mlp(model, node_features.cpu(), labels.cpu(), 
                                   val_mask, device)
            test_acc = evaluate_mlp(model, node_features.cpu(), labels.cpu(), 
                                    test_mask, device)
        
        epoch_time = time.time() - start_time
        
        history['train_loss'].append(loss)
        history['val_acc'].append(val_acc)
        history['test_acc'].append(test_acc)
        history['epoch_time'].append(epoch_time)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
        
        print(f"Epoch {epoch+1:2d}/{epochs} | Loss: {loss:.4f} | "
              f"Val: {val_acc:.4f} | Test: {test_acc:.4f} | Time: {epoch_time:.1f}s")
    
    print(f"\nBest validation accuracy: {best_val_acc:.4f}")
    print(f"Test accuracy at best val: {best_test_acc:.4f}")
    
    return history, best_val_acc, best_test_acc

print("Training wrapper defined")

Training wrapper defined


## Training Configuration

In [28]:
# Training hyperparameters
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT = 0.5
EPOCHS = 10
BATCH_SIZE = 1024
LR = 0.001

results = {}

print("Starting model comparison...")
print(f"\nConfiguration:")
print(f"  Hidden dimension: {HIDDEN_DIM}")
print(f"  Number of layers: {NUM_LAYERS}")
print(f"  Dropout: {DROPOUT}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LR}")

Starting model comparison...

Configuration:
  Hidden dimension: 256
  Number of layers: 2
  Dropout: 0.5
  Epochs: 10
  Batch size: 1024
  Learning rate: 0.001


## Train All Models

### 1. MLP Baseline

In [29]:
mlp_model = MLP(num_features, HIDDEN_DIM, num_classes, NUM_LAYERS, DROPOUT).to(device)
mlp_history, mlp_val, mlp_test = train_model(
    "MLP Baseline", mlp_model, is_gnn=False, 
    epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR
)
results['MLP'] = {'history': mlp_history, 'val_acc': mlp_val, 'test_acc': mlp_test}


Training MLP Baseline
Parameters: 37,935
Epoch  1/10 | Loss: 1.9699 | Val: 0.6104 | Test: 0.4880 | Time: 1.4s
Epoch  1/10 | Loss: 1.9699 | Val: 0.6104 | Test: 0.4880 | Time: 1.4s
Epoch  2/10 | Loss: 1.3672 | Val: 0.6481 | Test: 0.5219 | Time: 1.2s
Epoch  2/10 | Loss: 1.3672 | Val: 0.6481 | Test: 0.5219 | Time: 1.2s
Epoch  3/10 | Loss: 1.2801 | Val: 0.6632 | Test: 0.5350 | Time: 1.2s
Epoch  3/10 | Loss: 1.2801 | Val: 0.6632 | Test: 0.5350 | Time: 1.2s
Epoch  4/10 | Loss: 1.2325 | Val: 0.6759 | Test: 0.5464 | Time: 1.2s
Epoch  4/10 | Loss: 1.2325 | Val: 0.6759 | Test: 0.5464 | Time: 1.2s
Epoch  5/10 | Loss: 1.2021 | Val: 0.6811 | Test: 0.5489 | Time: 1.2s
Epoch  5/10 | Loss: 1.2021 | Val: 0.6811 | Test: 0.5489 | Time: 1.2s
Epoch  6/10 | Loss: 1.1780 | Val: 0.6935 | Test: 0.5590 | Time: 1.2s
Epoch  6/10 | Loss: 1.1780 | Val: 0.6935 | Test: 0.5590 | Time: 1.2s
Epoch  7/10 | Loss: 1.1632 | Val: 0.6964 | Test: 0.5613 | Time: 1.2s
Epoch  7/10 | Loss: 1.1632 | Val: 0.6964 | Test: 0.5613 | Tim

### 2. GraphSAGE (Mean)

In [21]:
sage_model = GraphSAGE(num_features, HIDDEN_DIM, num_classes, NUM_LAYERS, DROPOUT).to(device)
sage_history, sage_val, sage_test = train_model(
    "GraphSAGE (Mean)", sage_model, is_gnn=True, 
    epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR
)
results['GraphSAGE-Mean'] = {'history': sage_history, 'val_acc': sage_val, 'test_acc': sage_test}


Training GraphSAGE (Mean)
Parameters: 75,567
Epoch  1/10 | Loss: 1.0972 | Val: 0.8475 | Test: 0.6846 | Time: 2399.1s
Epoch  1/10 | Loss: 1.0972 | Val: 0.8475 | Test: 0.6846 | Time: 2399.1s
Epoch  2/10 | Loss: 0.5454 | Val: 0.8646 | Test: 0.7046 | Time: 2478.0s
Epoch  2/10 | Loss: 0.5454 | Val: 0.8646 | Test: 0.7046 | Time: 2478.0s
Epoch  3/10 | Loss: 0.4858 | Val: 0.8742 | Test: 0.7128 | Time: 2579.8s
Epoch  3/10 | Loss: 0.4858 | Val: 0.8742 | Test: 0.7128 | Time: 2579.8s
Epoch  4/10 | Loss: 0.4560 | Val: 0.8784 | Test: 0.7223 | Time: 2561.9s
Epoch  4/10 | Loss: 0.4560 | Val: 0.8784 | Test: 0.7223 | Time: 2561.9s
Epoch  5/10 | Loss: 0.4381 | Val: 0.8827 | Test: 0.7318 | Time: 2559.8s
Epoch  5/10 | Loss: 0.4381 | Val: 0.8827 | Test: 0.7318 | Time: 2559.8s
Epoch  6/10 | Loss: 0.4227 | Val: 0.8847 | Test: 0.7362 | Time: 2557.4s
Epoch  6/10 | Loss: 0.4227 | Val: 0.8847 | Test: 0.7362 | Time: 2557.4s
Epoch  7/10 | Loss: 0.4165 | Val: 0.8880 | Test: 0.7429 | Time: 2424.0s
Epoch  7/10 | Loss

### 3. GraphSAGE (MaxPool)

In [30]:
sage_max_model = GraphSAGEMaxPool(num_features, HIDDEN_DIM, num_classes, NUM_LAYERS, DROPOUT).to(device)
sage_max_history, sage_max_val, sage_max_test = train_model(
    "GraphSAGE (MaxPool)", sage_max_model, is_gnn=True, 
    epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR
)
results['GraphSAGE-MaxPool'] = {'history': sage_max_history, 'val_acc': sage_max_val, 'test_acc': sage_max_test}


Training GraphSAGE (MaxPool)
Parameters: 151,459
Epoch  1/10 | Loss: 1.1030 | Val: 0.8304 | Test: 0.6747 | Time: 2379.4s
Epoch  1/10 | Loss: 1.1030 | Val: 0.8304 | Test: 0.6747 | Time: 2379.4s
Epoch  2/10 | Loss: 0.5914 | Val: 0.8549 | Test: 0.6955 | Time: 2379.4s
Epoch  2/10 | Loss: 0.5914 | Val: 0.8549 | Test: 0.6955 | Time: 2379.4s
Epoch  3/10 | Loss: 0.5128 | Val: 0.8675 | Test: 0.7161 | Time: 2376.2s
Epoch  3/10 | Loss: 0.5128 | Val: 0.8675 | Test: 0.7161 | Time: 2376.2s
Epoch  4/10 | Loss: 0.4727 | Val: 0.8749 | Test: 0.7208 | Time: 2377.8s
Epoch  4/10 | Loss: 0.4727 | Val: 0.8749 | Test: 0.7208 | Time: 2377.8s
Epoch  5/10 | Loss: 0.4415 | Val: 0.8819 | Test: 0.7321 | Time: 2377.3s
Epoch  5/10 | Loss: 0.4415 | Val: 0.8819 | Test: 0.7321 | Time: 2377.3s
Epoch  6/10 | Loss: 0.4252 | Val: 0.8808 | Test: 0.7364 | Time: 2367.5s
Epoch  6/10 | Loss: 0.4252 | Val: 0.8808 | Test: 0.7364 | Time: 2367.5s
Epoch  7/10 | Loss: 0.4149 | Val: 0.8898 | Test: 0.7390 | Time: 2374.9s
Epoch  7/10 | 

### 4. GCN

In [31]:
gcn_model = GCN(num_features, HIDDEN_DIM, num_classes, NUM_LAYERS, DROPOUT).to(device)
gcn_history, gcn_val, gcn_test = train_model(
    "GCN", gcn_model, is_gnn=True, 
    epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR
)
results['GCN'] = {'history': gcn_history, 'val_acc': gcn_val, 'test_acc': gcn_test}


Training GCN
Parameters: 37,935


KeyboardInterrupt: 

### 5. GAT

In [ ]:
gat_model = GAT(num_features, HIDDEN_DIM, num_classes, NUM_LAYERS, DROPOUT).to(device)
gat_history, gat_val, gat_test = train_model(
    "GAT", gat_model, is_gnn=True, 
    epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR
)
results['GAT'] = {'history': gat_history, 'val_acc': gat_val, 'test_acc': gat_test}


Training GAT
Parameters: 38,238
Epoch  1/10 | Loss: nan | Val: 0.0583 | Test: 0.0455 | Time: 72562.3s
Epoch  1/10 | Loss: nan | Val: 0.0583 | Test: 0.0455 | Time: 72562.3s


## Results Summary

In [ ]:
# Create summary dataframe
summary_data = []
for model_name, data in results.items():
    summary_data.append({
        'Model': model_name,
        'Best Val Acc': data['val_acc'],
        'Test Acc': data['test_acc'],
        'Avg Epoch Time (s)': np.mean(data['history']['epoch_time']),
        'Final Train Loss': data['history']['train_loss'][-1]
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Test Acc', ascending=False)

print("\n" + "="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)
print(summary_df.to_string(index=False))
print("\n")

# Find best model
best_model = summary_df.iloc[0]
print(f"🏆 Best Model: {best_model['Model']}")
print(f"   Test Accuracy: {best_model['Test Acc']:.4f}")
print(f"   Validation Accuracy: {best_model['Best Val Acc']:.4f}")
print(f"   Avg Training Time: {best_model['Avg Epoch Time (s)']:.2f}s per epoch")

## Visualizations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Test Accuracy Comparison
ax1 = axes[0, 0]
models = summary_df['Model'].values
test_accs = summary_df['Test Acc'].values
colors = plt.cm.viridis(np.linspace(0, 1, len(models)))
ax1.barh(models, test_accs, color=colors)
ax1.set_xlabel('Test Accuracy')
ax1.set_title('Test Accuracy by Model')
ax1.grid(axis='x', alpha=0.3)

# Plot 2: Training Loss Curves
ax2 = axes[0, 1]
for model_name, data in results.items():
    ax2.plot(data['history']['train_loss'], label=model_name, marker='o')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Training Loss')
ax2.set_title('Training Loss Over Time')
ax2.legend()
ax2.grid(alpha=0.3)

# Plot 3: Validation Accuracy Curves
ax3 = axes[1, 0]
for model_name, data in results.items():
    ax3.plot(data['history']['val_acc'], label=model_name, marker='o')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Validation Accuracy')
ax3.set_title('Validation Accuracy Over Time')
ax3.legend()
ax3.grid(alpha=0.3)

# Plot 4: Accuracy vs Speed Trade-off
ax4 = axes[1, 1]
times = summary_df['Avg Epoch Time (s)'].values
accs = summary_df['Test Acc'].values
ax4.scatter(times, accs, s=200, alpha=0.6, c=range(len(models)), cmap='viridis')
for i, model in enumerate(models):
    ax4.annotate(model, (times[i], accs[i]), fontsize=9, alpha=0.8)
ax4.set_xlabel('Avg Training Time per Epoch (s)')
ax4.set_ylabel('Test Accuracy')
ax4.set_title('Accuracy vs Training Speed')
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('model_comparison_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization complete! Saved to model_comparison_results.png")

## Save Results

In [ ]:
# Save summary to CSV
summary_df.to_csv('model_comparison_results.csv', index=False)
print("Results saved to model_comparison_results.csv")

# Save best model
best_model_name = summary_df.iloc[0]['Model']
print(f"\nBest model: {best_model_name}")

## Detailed Analysis

In [ ]:
print("\n" + "="*80)
print("DETAILED ANALYSIS")
print("="*80)

# Improvement over baseline
mlp_acc = summary_df[summary_df['Model'] == 'MLP']['Test Acc'].values[0]
print(f"\nMLP Baseline Accuracy: {mlp_acc:.4f}")
print("\nImprovement over MLP baseline:")
for _, row in summary_df.iterrows():
    if row['Model'] != 'MLP':
        improvement = (row['Test Acc'] - mlp_acc) * 100
        print(f"  {row['Model']:20s}: +{improvement:.2f}% ({row['Test Acc']:.4f})")

# Speed comparison
print("\nTraining Speed:")
for _, row in summary_df.iterrows():
    print(f"  {row['Model']:20s}: {row['Avg Epoch Time (s)']:.2f}s per epoch")

# Convergence analysis
print("\nConvergence (epochs to reach 90% of final val accuracy):")
for model_name, data in results.items():
    final_val = data['val_acc']
    target = 0.9 * final_val
    val_history = data['history']['val_acc']
    epochs_to_converge = next((i+1 for i, v in enumerate(val_history) if v >= target), len(val_history))
    print(f"  {model_name:20s}: {epochs_to_converge} epochs")

## Recommendations

In [ ]:
print("\n" + "="*80)
print("RECOMMENDATIONS")
print("="*80)

best = summary_df.iloc[0]
fastest = summary_df.loc[summary_df['Avg Epoch Time (s)'].idxmin()]

print(f"\n✅ For best accuracy: Use {best['Model']}")
print(f"   - Test Accuracy: {best['Test Acc']:.4f}")
print(f"   - Training time: {best['Avg Epoch Time (s)']:.2f}s/epoch")

print(f"\n⚡ For fastest training: Use {fastest['Model']}")
print(f"   - Test Accuracy: {fastest['Test Acc']:.4f}")
print(f"   - Training time: {fastest['Avg Epoch Time (s)']:.2f}s/epoch")

print("\n📊 Key Findings:")
print(f"   - Graph structure provides {((best['Test Acc'] - mlp_acc) / mlp_acc * 100):.1f}% improvement")
print(f"   - {best['Model']} achieves the best performance")
print(f"   - All GNN models outperform the MLP baseline")

print("\n🔬 Next Steps:")
print("   1. Fine-tune hyperparameters for the best model")
print("   2. Try different neighbor sampling strategies")
print("   3. Analyze cold-start node performance specifically")
print("   4. Consider ensemble methods combining multiple models")